In [1]:
import requests
import pandas as pd
from datetime import datetime
import numpy as np
import scrapy
import asyncio

In [2]:
JGB_rates = pd.read_csv('https://www.mof.go.jp/jgbs/reference/interest_rate/data/jgbcm_all.csv', encoding='Shift-JIS', header=1)

In [3]:
JGB_rates.head()

,基準日,1年,2年,3年,4年,5年,6年,7年,8年,9年,10年,15年,20年,25年,30年,40年
0,S49.9.24,10.327,9.362,8.83,8.515,8.348,8.290,8.24,8.121,8.127,-,-,-,-,-,-
1,S49.9.25,10.333,9.364,8.831,8.516,8.348,8.290,8.24,8.121,8.127,-,-,-,-,-,-
2,S49.9.26,10.34,9.366,8.832,8.516,8.348,8.290,8.24,8.122,8.128,-,-,-,-,-,-
3,S49.9.27,10.347,9.367,8.833,8.517,8.349,8.290,8.24,8.122,8.128,-,-,-,-,-,-
4,S49.9.28,10.354,9.369,8.834,8.518,8.349,8.291,8.24,8.122,8.129,-,-,-,-,-,-


In [4]:
def dateConv(date):

    wareki = {
            "R":2018,
            "H":1988,
            "S":1925
            }
  
    date_els = date.split('.', 2)
    year_num = int(date_els[0][1:])
    era = date_els[0][0]
 
    year = year_num + wareki[era]

    month =int(date_els[1])
    day = int(date_els[2])

    dt = datetime(year, month, day)

    return dt



In [5]:
JGB_rates["基準日"] = JGB_rates['基準日'].apply(dateConv)
JGB_rates = JGB_rates.set_index('基準日')
JGB_rates.replace('-', np.nan, inplace=True)

In [6]:
JGB_rates.head()

,1年,2年,3年,4年,5年,6年,7年,8年,9年,10年,15年,20年,25年,30年,40年
基準日,,,,,,,,,,,,,,,
1974-09-24,10.327,9.362,8.83,8.515,8.348,8.290,8.24,8.121,8.127,NaN,NaN,NaN,NaN,NaN,NaN
1974-09-25,10.333,9.364,8.831,8.516,8.348,8.290,8.24,8.121,8.127,NaN,NaN,NaN,NaN,NaN,NaN
1974-09-26,10.34,9.366,8.832,8.516,8.348,8.290,8.24,8.122,8.128,NaN,NaN,NaN,NaN,NaN,NaN
1974-09-27,10.347,9.367,8.833,8.517,8.349,8.290,8.24,8.122,8.128,NaN,NaN,NaN,NaN,NaN,NaN
1974-09-28,10.354,9.369,8.834,8.518,8.349,8.291,8.24,8.122,8.129,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
JGB_rates.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 12517 entries, 1974-09-24 to 2023-06-30
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   1年      11869 non-null  object 
 1   2年      12156 non-null  object 
 2   3年      12441 non-null  object 
 3   4年      12517 non-null  float64
 4   5年      12517 non-null  float64
 5   6年      12517 non-null  float64
 6   7年      12517 non-null  float64
 7   8年      12517 non-null  float64
 8   9年      12517 non-null  float64
 9   10年     9156 non-null   object 
 10  15年     7815 non-null   object 
 11  20年     9043 non-null   object 
 12  25年     4722 non-null   object 
 13  30年     5840 non-null   object 
 14  40年     3828 non-null   object 
dtypes: float64(6), object(9)
memory usage: 1.5+ MB


In [ ]:
JGB_rates['5年'].resample('Y').mean()

In [9]:
JGB_rate_5_30 = JGB_rates[['5年', '10年', '15年', '20年', '25年', '30年']].dropna()
JGB_rate_5_30 = JGB_rate_5_30.astype(float, errors='raise')

In [10]:
JGB_rate_5_30_12M = JGB_rate_5_30.resample('M').mean().tail(12)
JGB_rate_5_30_12M

,5年,10年,15年,20年,25年,30年
基準日,,,,,,
2022-07-31,0.020400,0.231300,0.597400,0.890300,1.097400,1.238550
2022-08-31,0.001136,0.206818,0.551409,0.824909,1.015545,1.162591
2022-09-30,0.064750,0.269250,0.670050,0.951600,1.160450,1.311800
2022-10-31,0.090850,0.257150,0.769350,1.081450,1.298250,1.456100
2022-11-30,0.087100,0.265300,0.786750,1.104550,1.314900,1.480100
2022-12-31,0.178273,0.357045,0.873182,1.164773,1.351409,1.485909
2023-01-31,0.235947,0.479053,1.054789,1.327947,1.489789,1.575895
2023-02-28,0.214895,0.520053,1.047158,1.310474,1.444000,1.508579
2023-03-31,0.150727,0.402455,0.860500,1.135818,1.265727,1.353545


In [11]:
JGB_rate_5_30_12M.mean()

5年     0.117466
10年    0.358119
15年    0.798023
20年    1.077380
25年    1.249725
30年    1.368891
dtype: float64